## Benchmarks de PubMedBert y BioBert

En este notebook se realizan los benchmarks de PubMedBert y BioBert, para ello se obtendrán los datos de cada uno de los datasets y con la ayuda de `simpletransformers` se entrenarán y evaluarán los modelos, por último los resultaods se guardarán en un json.

### Instalación de librerías

In [ ]:
!pip install -q transformers datasets bioc simpletransformers

In [ ]:
from simpletransformers.ner import NERModel
from ner_benchmark_data_wrangler import NERBenchmarkDataWrangler
from datasets_utils import datasets_map
import json

### Carga de datos y benchmarks

In [ ]:
models = ["dmis-lab/biobert-v1.1", "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"]
results = {}

for model in models:
  for dataset in datasets_map:
    data_wrangler = NERBenchmarkDataWrangler(dataset, model, datasets_map)
    train_df = data_wrangler.get_format_data('train')
    valid_df = data_wrangler.get_format_data('valid')
    test_df = data_wrangler.get_format_data('test')

    train_args = {
      'reprocess_input_data': True,
      'overwrite_output_dir': True,
      'sliding_window': True,
      'max_seq_length': 64,
      'num_train_epochs': 20,
      'train_batch_size': 32,
      'fp16': True,
      'evaluate_during_training': True,
    }

    custom_labels = train_df['labels'].unique().tolist() + valid_df['labels'].unique().tolist() + test_df['labels'].unique().tolist()
    custom_labels = list(set(custom_labels))
    ner_model = NERModel('bert', model, labels=custom_labels, args=train_args)

    ner_model.train_model(train_df, eval_data=valid_df)

    result, model_outputs, preds_list = ner_model.eval_model(test_df)

    results[f'{model}_{dataset}'] = result

### Resultados

In [ ]:
json.dump(results, open('results.json', 'w'))
print(results)